In [1]:
#!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

In [2]:
#import torch

In [3]:
#torch.cuda.is_available()

In [4]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#print(device)


In [5]:
#!pip install facenet-pytorch

In [6]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from PIL import Image
import cv2
import time
import os

C:\Users\asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))


Running on device: cpu


In [8]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,keep_all=False,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

In [9]:
resnet = InceptionResnetV1(pretrained='vggface2').eval()

In [10]:
workers = 0 if os.name == 'nt' else 4

In [11]:
'''
#aplicar filtros a la imagen de entrenamiento, data augmentation
from PIL import Image, ImageDraw, ImageFilter

filters = {
    "Blur": ImageFilter.BLUR,
    "Contour": ImageFilter.CONTOUR,
    "Detail": ImageFilter.DETAIL,
    "Edge Enhance": ImageFilter.EDGE_ENHANCE,
    "Edge Enhance More": ImageFilter.EDGE_ENHANCE_MORE,
    "Emboss": ImageFilter.EMBOSS,
    "Find Edges": ImageFilter.FIND_EDGES,
    "Sharpen": ImageFilter.SHARPEN,
    "Smooth": ImageFilter.SMOOTH,
    "Smooth More": ImageFilter.SMOOTH_MORE,
    "Box Blur": ImageFilter.BoxBlur(10),
    "Gaussian Blur": ImageFilter.GaussianBlur(25),
    "Unsharp Mark": ImageFilter.UnsharpMask,
}

if __name__ == "__main__":
    for key, value in filters.items():
        with Image.open("./p/train/Jeremy_Meeks_Mug_Shot.jpg") as im:
            im = im.filter(value)
            im.save(f"./p/train/{key}.jpg")

'''



'\n#aplicar filtros a la imagen de entrenamiento, data augmentation\nfrom PIL import Image, ImageDraw, ImageFilter\n\nfilters = {\n    "Blur": ImageFilter.BLUR,\n    "Contour": ImageFilter.CONTOUR,\n    "Detail": ImageFilter.DETAIL,\n    "Edge Enhance": ImageFilter.EDGE_ENHANCE,\n    "Edge Enhance More": ImageFilter.EDGE_ENHANCE_MORE,\n    "Emboss": ImageFilter.EMBOSS,\n    "Find Edges": ImageFilter.FIND_EDGES,\n    "Sharpen": ImageFilter.SHARPEN,\n    "Smooth": ImageFilter.SMOOTH,\n    "Smooth More": ImageFilter.SMOOTH_MORE,\n    "Box Blur": ImageFilter.BoxBlur(10),\n    "Gaussian Blur": ImageFilter.GaussianBlur(25),\n    "Unsharp Mark": ImageFilter.UnsharpMask,\n}\n\nif __name__ == "__main__":\n    for key, value in filters.items():\n        with Image.open("./p/train/Jeremy_Meeks_Mug_Shot.jpg") as im:\n            im = im.filter(value)\n            im.save(f"./p/train/{key}.jpg")\n\n'

In [12]:

'''
def collate_fn(x):
    return x[0]

dataset = datasets.ImageFolder('./train')
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
loader = DataLoader(dataset, collate_fn=collate_fn, num_workers=workers)
'''


"\ndef collate_fn(x):\n    return x[0]\n\ndataset = datasets.ImageFolder('./train')\ndataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}\nloader = DataLoader(dataset, collate_fn=collate_fn, num_workers=workers)\n"

In [13]:
'''
name_list=[]
embedding_list=[]
for img, idx in loader:
    face, prob = mtcnn(img, return_prob=True) 
    if face is not None and prob>0.92:
        emb = resnet(face.unsqueeze(0)).to(device) 
        embedding_list.append(emb.detach()) 
        name_list.append(dataset.idx_to_class[idx]) 

data = [embedding_list, name_list] 
torch.save(data, 'data.pt') # saving data.pt file
'''



"\nname_list=[]\nembedding_list=[]\nfor img, idx in loader:\n    face, prob = mtcnn(img, return_prob=True) \n    if face is not None and prob>0.92:\n        emb = resnet(face.unsqueeze(0)).to(device) \n        embedding_list.append(emb.detach()) \n        name_list.append(dataset.idx_to_class[idx]) \n\ndata = [embedding_list, name_list] \ntorch.save(data, 'data.pt') # saving data.pt file\n"

In [14]:
load_data=torch.load('data.pt')
embedding_list=load_data[0]
name_list=load_data[1]


In [15]:
'''
img=cv2.imread('./val/Davis Alderete/310.png')
img_cropped,prob=mtcnn(img,return_prob=True)


boxes,_=mtcnn.detect(img)

if prob>0.90:
    img_embedding=resnet(img_cropped.unsqueeze(0)).to(device)
    dist_list=[]
    for idx,emb_db in enumerate(embedding_list):
        dist=torch.dist(img_embedding,emb_db).item()
        dist_list.append(dist)
    min_dist=min(dist_list)
    min_dist_idx=dist_list.index(min_dist)
    name=name_list[min_dist_idx]

    box=boxes[0]
    original_img=img.copy()
    if min_dist<0.90:
        img=cv2.putText(img,name+' '+str(min_dist),(int(box[0]),int(box[1])),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),1,cv2.LINE_AA)
    img=cv2.rectangle(img,(int(box[0]),int(box[1])),(int(box[2]),int(box[3])),(255,0,0),2)



while True:
    cv2.imshow("Smarcity ",img)
    cv2.waitKey(0)
    #sys.exit() # to exit from all the processes
 
cv2.destroyAllWindows() # destroy all windows

'''

    


'\nimg=cv2.imread(\'./val/Davis Alderete/310.png\')\nimg_cropped,prob=mtcnn(img,return_prob=True)\n\n\nboxes,_=mtcnn.detect(img)\n\nif prob>0.90:\n    img_embedding=resnet(img_cropped.unsqueeze(0)).to(device)\n    dist_list=[]\n    for idx,emb_db in enumerate(embedding_list):\n        dist=torch.dist(img_embedding,emb_db).item()\n        dist_list.append(dist)\n    min_dist=min(dist_list)\n    min_dist_idx=dist_list.index(min_dist)\n    name=name_list[min_dist_idx]\n\n    box=boxes[0]\n    original_img=img.copy()\n    if min_dist<0.90:\n        img=cv2.putText(img,name+\' \'+str(min_dist),(int(box[0]),int(box[1])),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),1,cv2.LINE_AA)\n    img=cv2.rectangle(img,(int(box[0]),int(box[1])),(int(box[2]),int(box[3])),(255,0,0),2)\n\n\n\nwhile True:\n    cv2.imshow("Smarcity ",img)\n    cv2.waitKey(0)\n    #sys.exit() # to exit from all the processes\n \ncv2.destroyAllWindows() # destroy all windows\n\n'

In [16]:
def recognition(img,emb_list):
    
    img_cropped_list,prob=mtcnn(img,return_prob=True)
    
    boxes,_=mtcnn.detect(img)
    if img_cropped_list is not None:

        if prob>0.80:
                img_embedding=resnet(img_cropped_list.unsqueeze(0)).detach()
                dist_list=[]
                    
                for idx,emb_db in enumerate(emb_list):
                    dist=torch.dist(img_embedding,emb_db).item()
                    dist_list.append(dist)
                min_dist=min(dist_list)
                min_dist_idx=dist_list.index(min_dist)
                name=name_list[min_dist_idx]

                box=boxes[0]
                original_img=img.copy()
                if min_dist<0.90:
                    img=cv2.putText(img,name+' '+str(min_dist),(int(box[0]),int(box[1])),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),1,cv2.LINE_AA)
                img=cv2.rectangle(img,(int(box[0]),int(box[1])),(int(box[2]),int(box[3])),(255,0,0),2)
        
    return img

In [17]:
##webcam implemented
import cv2

capture = cv2.VideoCapture(0)

while True:
    ret, frame = capture.read()
    if not ret:
        print("fail to grab frame, try again")
        break
    img=recognition(frame,embedding_list)
    cv2.imshow('webCam',img)
    if cv2.waitKey(20)==27:
        break

capture.release()
cv2.destroyAllWindows()



C:\Users\asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


In [18]:

'''
aligned = torch.stack(aligned).to(device)
embeddings = resnet(aligned).detach().cpu()
'''




'\naligned = torch.stack(aligned).to(device)\nembeddings = resnet(aligned).detach().cpu()\n'

In [19]:
'''
import pandas as pd
dists = [[(e1 - e2).norm().item() for e2 in embeddings] for e1 in embeddings]
pd.DataFrame(dists, columns=names, index=names)

'''



'\nimport pandas as pd\ndists = [[(e1 - e2).norm().item() for e2 in embeddings] for e1 in embeddings]\npd.DataFrame(dists, columns=names, index=names)\n\n'